In [2]:
!pip install python-dotenv

In [ ]:
# https://docs.signnow.com/docs/signnow/document/operations/update-a-v-2-document-prefill-text

In [3]:
import requests
import json
from datetime import datetime,timedelta
from dotenv import load_dotenv
import functools
import os

In [4]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
load_dotenv()

True

In [6]:
os.getenv('client_id')

'e12119018f2e0ca97cfef13800c0f030'

In [7]:
#test form details
signer_count = 2
user_details = [{'email': 'mskishan26@gmail.com', 'name': 'Kishan Sathish Babu'},
                {'email': 'obinsontobinson@gmail.com', 'name': 'Obinson Tobinson'}]
house_name = '1234 xyz st'
movein_date = '09-01-2024'
rent = '3000'

In [8]:
all_names = [person['name'] for person in user_details]

def all_tenant_prefill(curr_name):
  other_names = [name for name in all_names if name != curr_name]
  return f"{curr_name}, {','.join(other_names)}"

In [9]:
class SignNowAPI:
    def __init__(self):
        self.base_url = "https://api.signnow.com"
        self.token = self.get_access_token()

    def get_access_token(self):
        url = f"{self.base_url}/oauth2/token"
        payload = {
            "grant_type": "password",
            "client_id": os.getenv('client_id'),
            "client_secret": os.getenv('client_secret'),
            "username": os.getenv('username'),
            "password": os.getenv('password')
        }
        response = requests.post(url, data=payload)
        response.raise_for_status()
        token_data = response.json()
        self.token = token_data["access_token"]
        self.token_expiry = datetime.now() + timedelta(seconds=token_data["expires_in"])
        logger.info("New token issued")

    def verify_token(self):
        if not self.token or datetime.now() >= self.token_expiry:
            return False
        return True

    def token_required(func):
        @functools.wraps(func)
        def wrapper(self, *args, **kwargs):
            if not self.verify_token():
                logger.info("Token expired or not available. Refreshing...")
                self.get_access_token()
            return func(self, *args, **kwargs)
        return wrapper

    @token_required
    def create_document_group(self, document_list, group_name):
        url = f"{self.base_url}/documentgroup"
        headers = {
            "Authorization": f"Bearer {self.token}",
            "Content-Type": "application/json"
        }
        payload = {"document_ids": document_list,
                   "group_name":group_name}
        response = requests.post(url, headers=headers, json=payload)
        return response.json()["id"]

    @token_required
    def create_document_from_template(self, template_id, document_name):
        url = f"{self.base_url}/template/{template_id}/copy"
        headers = {
            "Authorization": f"Bearer {self.token}",
            "Content-Type": "application/json"
        }
        payload = {"document_name": document_name}
        response = requests.post(url, headers=headers, json=payload)
        return response.json()["id"]

    @token_required
    def send_invite(self, documentgroup_id, payload):
        url = f"{self.base_url}/documentgroup/{documentgroup_id}/groupinvite"
        headers = {
            "Authorization": f"Bearer {self.token}",
            "Content-Type": "application/json"
        }
        response = requests.post(url, headers=headers, json=payload)
        if response.status_code != 200:
          print(response.json())
        return response.status_code == 200

    @token_required
    def prefill_document(self, document_id, prefill_text):
        url = f"{self.base_url}/v2/documents/{document_id}/prefill-texts"
        headers = {
            "Authorization": f"Bearer {self.token}",
            "Content-Type": "application/json"
        }
        payload = {
            "fields": prefill_text
        }
        response = requests.put(url, headers=headers, json=payload)
        if response.status_code != 204:
          print(response.json())
        return response.status_code == 204

In [12]:
api = SignNowAPI()
api.get_access_token()
api.token

'0b9a974b95f525c27bcf5547b8bb76f24e10ee92d7e6f7d5488ae11baf2d6f13'

In [10]:
#init the signnow class
api = SignNowAPI()
template_id = os.getenv('template_id')

#create all the documents from the template and store in a list
#the number of tenants(users) can be predetermined from the no of bedrooms in the house while filling the application request
#details like move in date, rent due and address can be checked and modified before creating the document package and sending the invites
document_list = []
invite_json = {}
invite_list = []
text_field_dict = {}
invite_subject = f"Please sign the application for {house_name}"
order = [1,2,2]
for signer in range(len(user_details)):
  #doc creation
  doc_name = f"{house_name}, {user_details[signer]['name']}"
  doc_id = api.create_document_from_template(template_id, doc_name)
  text_field_list = []
  text_field_list.append({"field_name":'Text Field 122',"prefilled_text":  all_tenant_prefill(user_details[signer]['name'])}) #all tenants line
  text_field_list.append({"field_name":'Text Field 91',"prefilled_text":  house_name}) #house address
  text_field_list.append({"field_name":'Text Field 109',"prefilled_text":  rent}) #rent
  prefill_status = api.prefill_document(doc_id, text_field_list)
  if not prefill_status:
    print('Prefill error')
  document_list.append(doc_id)

  #invite generation
  curr_invite = {}
  curr_invite['order'] = signer+1

  curr_invite_mail = {'email':user_details[signer]['email'], 'subject': invite_subject, 'expiration_days':7}
  curr_invite['invite_emails'] = [curr_invite_mail]

  curr_invite_action = {'email':user_details[signer]['email'],
                         'role_name': 'Recipient 1',
                         'action': 'sign',
                         'document_id': doc_id,
                         'allow_reassign': '0',
                         'decline_by_signature': '0'}
  curr_invite['invite_actions'] = [curr_invite_action]

  invite_list.append(curr_invite)

completion_emails = [{'email' : os.getenv('username'),
                      'subject': f"Singing completed for {house_name}",
                      'message': 'document group signing completed'}]

invite_json['invite_steps'] = invite_list
invite_json['completion_emails'] = completion_emails



# Create a new document group
group_name = f"{house_name}, {user_details[0]['name']}, {datetime.now().strftime('%m-%d-%Y')}"
documentgroup_id = api.create_document_group(document_list, group_name)

# # Send an invite to sign the document
api.send_invite(documentgroup_id, invite_json)

print(f"New folder created: {documentgroup_id}")

{'errors': [{'code': 65582, 'message': 'An invite_step is missing an order attribute or the attribute is invalid. Must be between 1 and 100'}]}
New folder created: 3045d39375494592b82221e3d060d01b09d006e4


In [ ]:
# #we can use this for setting a few links after the signing!

# "redirect_uri": "https://example.com",
# "decline_redirect_uri": "https://signnow.com",
# "close_redirect_uri": "https://close-redirect-uri.com",
# "redirect_target": "blank"

In [12]:
api.token

'3441b784a2f2870f9d077aed119da80a99fbbef4e69d7f323c0c342f663be55f'

In [11]:
invite_json

{'invite_steps': [{'order': 0,
   'invite_emails': [{'email': 'mskishan26@gmail.com',
     'subject': 'Please sign the application for 1234 xyz st',
     'expiration_days': 7}],
   'invite_actions': [{'email': 'mskishan26@gmail.com',
     'role_name': 'Recipient 1',
     'action': 'sign',
     'document_id': '11268185c3fc4b82bf8fd73beabfd5720b2038bc',
     'allow_reassign': '0',
     'decline_by_signature': '0'}]},
  {'order': 1,
   'invite_emails': [{'email': 'obinsontobinson@gmail.com',
     'subject': 'Please sign the application for 1234 xyz st',
     'expiration_days': 7}],
   'invite_actions': [{'email': 'obinsontobinson@gmail.com',
     'role_name': 'Recipient 1',
     'action': 'sign',
     'document_id': 'c5af5992af1440b29d6055138f582d968952967d',
     'allow_reassign': '0',
     'decline_by_signature': '0'}]}],
 'completion_emails': [{'email': 'sathishbabu.ki@northeastern.edu',
   'subject': 'Singing completed for 1234 xyz st',
   'message': 'document group signing completed'